# ETL Pipeline Preparation
## 1. Import libraries and load datasets

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

# Output of the kaggle data sources
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# load messages dataset
messages = pd.read_csv("/kaggle/input/disasterresponse/disaster_messages.csv")
messages.head()

In [ ]:
# load categories dataset
categories = pd.read_csv("/kaggle/input/disasterresponse/disaster_categories.csv")
categories.head()

In [ ]:
# take a look at the shape of each dataset
print(messages.shape)
print(categories.shape)

## 2. Combine Datasets

In [ ]:
#  Merge the datasets together based on their common column 'id'
df = messages.merge(categories, on='id')
df.head()

## 3. Split categories into separate category columns

In [ ]:
# Create a new dataset of the 36 individual category columns and separate the solumns based on the charakter ;
categories = df['categories'].str.split(";",expand = True)
categories.head()

In [ ]:
# Select the first row of the categories dataframe
row = categories.iloc[0,:].values

# Use this row to extract a list of new column names for categories
category_colnames = [r[:-2] for r in row]
print(category_colnames)

In [ ]:
# Rename the columns of `categories`
categories.columns = category_colnames
categories.head()

## 4. Convert category values to numbers 0 or 1

In [ ]:
# Convert category values to just numbers 0 or 1
for column in categories:
 
    # Set each value to be only the last character of the string
    categories[column] = categories[column].str[-1]
    
    # Convert column from string to numeric
    categories[column] = pd.to_numeric(categories[column])
    
categories.head()

## 5. Replace categories column in the original dataset with new category columns

In [ ]:
# Drop the original categories column from `df`
df.drop('categories', axis = 1, inplace = True)
df.head()

In [ ]:
# Fill in the new `categories` columns
df = pd.concat([df, categories], axis=1)
df.head()

## 6. Remove duplicates

In [ ]:
# Check the number of duplicates
df.duplicated().sum()

In [ ]:
# Drop the duplicates
df.drop_duplicates(inplace = True)

In [ ]:
# Check the number of duplicates agein
df.duplicated().sum()

## 7. Save the clean dataset into an sqlite database

In [ ]:
# Using pandas to_sql method to create an sqlite database
engine = create_engine('sqlite:///DisasterResponse.db')
df.to_sql('DisasterResponse', engine, index=False)

## 8. With this notebook complete process_data.py
For the result, see workspace/data/process_data.py